<a href="https://colab.research.google.com/github/yuyunliu33/MAIS202-MLbootcamp/blob/master/MAIS202_Assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 2 ― Classification

## 0. Introduction

In this second assigment, we will explore another cornerstone of machine learning: supervised classification. We will be specifically classifying movie reviews by their corresponding score. To do this, we will first pre-process the raw data by cleaning and turning each review into a vector. Then, we will explore the following learning algorithms for classification: support vector classifiers, random forests, and naive Bayes classifiers.

* [Question 1.1](#scrollTo=qQPEFaRiRbOS)
* [Question 2.1](#scrollTo=LawTBDrOcPJe)
* [Question 2.2](#scrollTo=Hqp7LyWtlLVx)
* [Question 3.1](#scrollTo=Qs_t1uVzh01s)
* [Question 3.2](#scrollTo=fl9Z0VVXoRsY)
* [Question 3.3](#scrollTo=Perywb8YapEU)
* [Question 4.1](#scrollTo=YKLBuWjmAKoJ)
* [Question 5.1](#scrollTo=Myn-42J9ACsH)
* [Question 6.1](#scrollTo=fN-dse1NBQnk)
* [Question 6.2](#scrollTo=wfCjr-JrEJya)
* [Quesiton 6.3](#scrollTo=l1iGVZtkE5fF)
$% latex commands for later use$
$\newcommand{\R}{\mathbb{R}}$
$\newcommand{\B}{\mathbb{B}}$
$\newcommand{\argmax}{\operatorname*{arg\ max}}$
$\newcommand{\given}{\; \vert \;}$



## 1. Importing Libraries and Data

For this assignment, we will be using a dataset of metatritic reviews. The data consists of a csv file where the first column is a string containing a user review and the second column contains the corresponding score that the user gave to the movie. First, we will import any libraries that we might use.

**Note.** You may use any library you'd like unless when otherwise specified to not use any or to use a particular one.

### Question 1.1 Importing Libraries

Add any modules you use as you complete the assignment here.

In [0]:
import csv
import random
### Answer starts here ###       
import copy
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score 

### Answer ends here ###

In [0]:
!wget https://raw.githubusercontent.com/LiTigre/McGillAI-asg2-data/master/metacritic_dataset.csv

--2020-02-02 17:25:05--  https://raw.githubusercontent.com/LiTigre/McGillAI-asg2-data/master/metacritic_dataset.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 22960821 (22M) [text/plain]
Saving to: ‘metacritic_dataset.csv.1’

metacritic_dataset. 100%[===================>]  21.90M   142MB/s    in 0.2s    

2020-02-02 17:25:05 (142 MB/s) - ‘metacritic_dataset.csv.1’ saved [22960821/22960821]



Let's create a variable to hold the name of the dataset for convenience.

In [0]:
dataset_filename = 'metacritic_dataset.csv'

Also create a function to print a review.

In [0]:
def print_review(review, score):
  print('--------------- Review with score of {} ---------------'.format(score))
  print(review)
  print('------------------------------------------------------')
  print()
  

Let's load the data and see what the first 10 reviews look like.

In [0]:
with open(dataset_filename) as csv_file:
  csv_reader = csv.reader(csv_file)
  colnames = next(csv_reader)
  data = list(csv_reader)
  
for review, score in random.sample(data, 10):
  print_review(review, score)

--------------- Review with score of 9 ---------------
Totally involved story of the continuity inherent in Brazilian urban slum crime. Every character is disposable in their roles in this story, which is otherwise totally mesmerizing.
------------------------------------------------------

--------------- Review with score of 10 ---------------
Maybe not the biggest marvel movie yet but definable the most enjoyable one, not going to lie this movie surprised me, it is prove that bigger isn't always better a simple plot of a heist was very enjoyable. Paul Rudd was surprisingly good as the hero,excellent time into the MCU. Looking forward to seeing more of ant man in the future
------------------------------------------------------

--------------- Review with score of 8 ---------------
Fun, Fun, Fun. I left loving ABBA all over again. However, I do question the casting of Pierce Brosnan??
------------------------------------------------------

--------------- Review with score of 8 ----

## 2. Preprocessing

 We will be converting our data into a binary bag-of-words representation (Google "binary bag-of-words"). To do this, we will perform two steps beforehand.

### Question 2.1 Cleaning the Data
Create a function called `clean`, which takes a string and then:

 1. lower-case all words 
 2. only keeps letters and spaces
  
  
  For example, this will cause
  
  >`This was the WORST movie I have EVER SEEN!!!!!!`
  
  to become
  
  >`this was the worst movie i have ever seen`
  
   Of course, you could do more pre-processing steps if you would like, such as lemmatization, stemming, etc... but TOTALLY OPTIONAL!

In [0]:
def clean(review):
  ### Start of Answer ###
  result = ""

  review = review.lower()
  
  for e in review:
    if e.isalpha() or e == " ":
      result += e

  return result 
  ### End of Answer ###


Test your function with this example string

In [0]:
print(clean("This was the WORST movie I have EVER SEEN!!!!!!"))

this was the worst movie i have ever seen


Now, we'll use the function to clean the whole dataset. We'll also turn the scores into integers while we're at it.

In [0]:
clean_data = [[clean(review), int(score)] for review, score in data]

for review, score in random.sample(clean_data, 5):
  print_review(review, score)

--------------- Review with score of 5 ---------------
i thought the movie was boring there wasnt really anything that grabbed my full attention
------------------------------------------------------

--------------- Review with score of 10 ---------------
the combination of comedy romance and drama is as good as it can be the movie definitely deals more with the power of familys love in the middle of the tragedy of war this movie doesnt intend to retell what the holocaust is about if some critics want that well go and rent schindlers list again
------------------------------------------------------

--------------- Review with score of 10 ---------------
this movie was so much fun because it was so funny the good cop bad routine was a riot will farrell was at his best in this movie he should do more kid movies
------------------------------------------------------

--------------- Review with score of 10 ---------------
one of or better yet the best comic book movie ever expect a real

In [0]:
print(clean_data[:5])

[['pertaining everything from story depth to animation pixar has created a movie that is considered a true cinematic masterpiece the movie of my life cheers pixar', 10], ['this is my favourite movie of all time i love the characters i love the story i mean this movie is just brilliantthe animation is remarkablebrilliant just brilliant', 10], ['original and just lovely toy story is a fantastic film that will entertain many from any age alike and it is just a great start to pixars full length films i give this movie ', 10], ['toy story is a fantastic movie with witty jokes good characters a great story my favourite movie of all time its basically the definition of a perfect movie', 10], ['unless youve been living under a rock for over  years you know about this movie i mean who doesnt toy story was the first featurelength film made completely with computer animation it also has a voice cast with stars we all have heard of like tom hanks and tim allen the movie even won john lasseter the 

### Question 2.2 Picking features

We now need to turn each review into vectors. We will pick the top 2000 words as features

Using those 2000 features, create a function called `vectorize` which will take a string as an input, and convert it to a vector using the binary bag of words representation.

For example, the string `"This movie made me cry"` will become a vector of size 2000 with 5 elements being 1 (assuming each word is part of the 2000 most common) and 1995 being 0, that it, is i will look something like

 > `[0, 0, ..., 0, 1, 0, ..., 0, 1, 0..., 0, 1, 0, ..., 0, 1, 0 ..., 0, 1, 0, ..., 0, 0]`
 
 In order to accomplish this task, you will
 
 1. write a `get_vocab` function which takes as an argument a list of (cleaned) reviews and the vocabulary size and outputs the a list of size `vocab_size` containing the most common words.
 2. write a `vectorize` function which takes as an argument a review and the vocabulary and turns the review into its binary bag of words representation.
 3. use the `vectorize` function to create a new variable called `vectorized_data` which will contain the bag-of-words representation of each data point contained in the `clean_data` variable rather than its string representation. Don't forget that a data point consists of review-score pairs.

**Warning**: this may take up to five minutes depending on implementation. Make use of dictionaries if you want this to be faster. However, speed is not evaluated.

In [0]:
def get_vocab(reviews, vocab_size):
  # first count the occurence of all the words in the data 
  # create an empty dict to store the occurences of each word
  d = {}
  for review in reviews:
    tmp = review.split()
    for word in tmp:
      if word not in d:
        d[word] = 1
      else:
        d[word] += 1

  # now that we have the occurences of the words, let's that the first 2000 most common words from the dict
  sorted_list = sorted(((v, k) for k, v in d.items()), reverse=True)[:2000] # get the first 2000 words
  first_2000 = []
  for w in sorted_list:
    first_2000.append(w[1])
  
  return first_2000


Test your function with the following code.
The `vocabulary` variable should have a length of 2000 and the most common words should be "the", "and", "a", etc.

In [0]:
num_features = 2000
vocabulary = get_vocab([review for review, score in clean_data], num_features)
print(len(vocabulary))
print(vocabulary)

2000
['the', 'and', 'a', 'of', 'to', 'is', 'it', 'i', 'this', 'movie', 'in', 'that', 'was', 'but', 'for', 'film', 'with', 'as', 'you', 'not', 'its', 'are', 'be', 'on', 'have', 'one', 'all', 'like', 'just', 'an', 'good', 'so', 'at', 'from', 'they', 'great', 'story', 'by', 'his', 'more', 'if', 'what', 'he', 'has', 'some', 'about', 'very', 'really', 'or', 'there', 'out', 'my', 'me', 'best', 'movies', 'who', 'time', 'see', 'action', 'no', 'characters', 'were', 'than', 'will', 'much', 'well', 'up', 'even', 'first', 'would', 'can', 'when', 'which', 'only', 'dont', 'how', 'had', 'most', 'because', 'plot', 'we', 'also', 'character', 'people', 'too', 'do', 'into', 'films', 'get', 'their', 'way', 'bad', 'been', 'other', 'better', 'make', 'did', 'think', 'made', 'watch', 'acting', 'seen', 'could', 'love', 'scenes', 'still', 'ever', 'her', 'any', 'many', 'being', 'them', 'then', 'your', 'does', 'end', 'say', 'after', 'didnt', 'never', 'spaghetti', 'moms', 'little', 'while', 'go', 'know', 'over', '

In [0]:
def vectorize(some_string, vocab):
  ### Answer starts here ###

  # get a list of words from the review (some_string)
  tmp = some_string.split()
  vector = vocab.copy()

  for i, word in enumerate(vector):
    if word in tmp:
      vector[i] = 1
    else:
      vector[i] = 0

  return vector
  ### Answer ends here ###

Test your function with the following input.
The vector should have four "1"s.

In [0]:
vector = vectorize("the and a of zyxw", vocabulary)
print(vector)
print(sum(vector))

[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

Now, vectorize the whole dataset.

In [0]:
### Answer starts here ###
vectorized_data = []

for review, score in clean_data:
  vectorized_review = vectorize(review, vocabulary)
  vectorized_data.append([vectorized_review, score])
### Answer ends here ###

In [0]:
for vector, score in random.sample(vectorized_data, 5):
  print_review(vector, score)

--------------- Review with score of 10 ---------------
[1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0

For convenience, we will write a function called `preprocess_sample_point` which takes as input a single raw review and ouputs its binary bag-of-words representation.

In [0]:
def preprocess_sample_point(review, vocab):
  return vectorize(clean(review), vocab)

In [0]:
vectorized_review = preprocess_sample_point(
    'The movie was not bad, it was really good!', vocabulary)
print(sum(vectorized_review))
print(vectorized_review)

8
[1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

### Question 2.4 Cleaning, Again

You may have noticed that some of the reviews are not in English. If we keep them in the dataset, this may cause accuracy to decrease. This is due to the fact that many of them will contain no words from our vocabulary, and will all have a zero-vector bag-of-words representation. However, there may be non-English language reviews that contain at least one word which is spelled the same as an English word. So, a better heuristic to determine if a review is in English or not is to check if it has at least 5 words from our vocabulary.

To that end, create a variable called `english_data`, which contains only the bag-of-words representations of english reviews by keeping only those which have at least 5 non-zero entries.



In [0]:
### Answer starts here ### 
english_data = []

for vector, score in vectorized_data:
  if sum(vector) >= 5:
    english_data.append([vector, score])

### Answer ends here ###

In [0]:
print(len(vectorized_data), len(english_data))

39758 38928


## 3. Train-Test Split

### Question 3.1 Shuffling

The dataset is not ordered randomly. In fact, reviews from the same movie are put one after another as well as movies from the same year. To avoid introducing bias when splitting the dataset, we should shuffle it.

To that end, first create a copy of the variable `english_data`, which will be called `shuffled_data`, and then shuffle it so that it consists of a random reordering of the rows in `english_data`. 

**Note**: In practice, making a copy is not necessary. This is done to make testing easier since Jupyter alters global state whenever a cell is executed.

In [0]:
# We create a copy of the clean_data
### Answer starts here ###

shuffled_data = copy.deepcopy(english_data)

### Answer ends here ###

# We will check the first 5 reviews, non-randomized
for review, score in english_data[0:5]:
  print_review(review, score)

--------------- Review with score of 10 ---------------
[1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [0]:
# Randomly reorder `shuffled_data`, making sure that `clean_data` is not altered
### Answer starts here ###
random.shuffle(shuffled_data)

### Answer ends here ###

In [0]:
# Let's check the first 5 movies in the randomized list
for review, score in shuffled_data[0:5]:
  print_review(review, score)

--------------- Review with score of 0 ---------------
[1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

### Question 3.2 Creating Feature and Class variables

Now that the data is shuffled, we will create a variable that will contain all the features, called `X` and another which will contain all the classes, called `y`.

Further, to simplify the task, we will use 2 classes intead of 11. That is, review scores 0, 1, 2, 3 will be class 0, review scores 9, 10 will be given class 1, and everything else will be ignored.

In [0]:
### Answer start here ###
X = [] # features
y = [] # classses

for vector, score in shuffled_data:
  if score in [0, 1, 2, 3]:
    y.append(0)
    X.append(vector)
  elif score in [9, 10]:
    y.append(1)
    X.append(vector)

### Answer ends here ###

Let's check the class balance

In [0]:
for i in range(2):
  print('class {}:'.format(i), 
        100 * len([0 for value in y if value == i])/len(y))


class 0: 32.434643865319316
class 1: 67.56535613468068


We have to keep in mind that most of the dataset is made out of labels which are 1. For instance, if a classifier always outputs 1, they would have an accuracy of 67%. Thus, 67% should be our baseline performance, not 50%.

### Question 3.3 Spliting the Dataset

Partition the dataset into a test set and a training dataset. Select 80% of the data to be part of the training set and the rest to be part of the test set.

To that end, write a function called `train_test_split` which
takes as input `X`, `y` and `test_size`, a floating point number between 0 and 1indicating how much of the dataset should be part of the test set. The function should return a tuple in the order:

`(X_train, X_test, y_train, y_test)`

In [0]:
print(round(len(y) * 0.8))
print(len(y)* 0.20)

19554
4888.6


In [0]:
def train_test_split(X, y, train_size):
  ### Answer starts here ###
  
  # get the size to partition train and test: 80% train, 20% test
  size = round(len(y) * train_size)

  X_train = X[:size]
  X_test = X[size:]
  y_train = y[:size]
  y_test = y[size:]
  
  return (X_train, X_test, y_train, y_test)

  ### Answer ends here ###

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 0.8)

In [0]:
y_test[:10]

[0, 1, 1, 1, 1, 0, 1, 0, 1, 1]

## 4. Support Vector Machines

A support vector classifier tries to find the best separating hyperplane through the data. If the data is linearly separable, it finds the a hyperplane which maximized the margin. If it isn't it tries to minimize cost associated with misclassifying points.



### Question 4.1 Creating a Support Vector Classifier
Using `scikit-learn`, create a support vector classifier for our review data.

1. Use `scikit-learn` to create a linear support vector classifer
2. Fit the model to our training set
3. Print training accuracy
4. Print test accuracy

In [0]:
### Answer starts here ###

# create a linear support vector classifier
svm_clf = LinearSVC()
svm_clf.fit(X_train, y_train)

# print test/training accuracy 
print(svm_clf.score(X_train, y_train))
print(svm_clf.score(X_test, y_test))

### Answer ends here ###

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


0.9531042241996522
0.8975250562487216


In [0]:
print(svm_clf.predict([preprocess_sample_point(
    'Boring. Such a bad movie. It was terrible and predictable', vocabulary)]))

print(svm_clf.predict([preprocess_sample_point(
    'I really liked this movie, it\'s great!', vocabulary)]))

[0]
[1]


## 5. Random Forests

Random forests are a kind of ensemble classifier, i.e. they are made up of a number of 'weak' learners where the final classification is a combination of the classifications of each learner. 



### Question 5.1 Creating a Random Forest Classifier
Using `scikit-learn`, create a radom forest classifier for our review data.

1. Use `scikit-learn` to create a random forest classifier
2. Fit the model to our training set
3. Print training accuracy
4. Print test accuracy



Be sure to check the [documentation](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html). Try to play around with the hyperparameters to see if you can get higher accuracy. Specifically, try finding good values for `n_estimators`, `min_samples_split`, `max_depth` and `max_features`. Try to beat the linear SVM's accuracy.

In [0]:
### Answer starts here ###

rfc = RandomForestClassifier()
rfc.fit(X_train, y_train)

print(rfc.score(X_train, y_train))
print(rfc.score(X_test, y_test))

### Answer ends here ###

1.0
0.8803436285538965


In [0]:
print(rfc.predict([preprocess_sample_point(
    'Boring. This movie is terrible', vocabulary)]))

print(rfc.predict([preprocess_sample_point(
    'This movie was pretty good', vocabulary)]))

[0]
[1]


##6. Naive Bayes
Until now, we've used pre-existing implementations of two types of classifiers: random forests and support vector machines. Now, we will cover a third type of classifier: naive Bayes classifers. However, this time we will be writing it from scratch.

Naive Bayes classifiers are part of a larger family of classifiers which are so called 'probabilistic classifiers'. This is because they do not only try to predict classes given features, but they estimate probabilities distributions over a set of classes.

First, we will go over some definitions.

**Definition.** A *prior probability* is the likelihood of an event given no further assumptions. For instance, the probability that it's raining is relatively low.

**Definiton.** A *posterior probability* or *conditional probability* is the likelihood of an event given that some other event has occurred. For instance, the probability that it's raining given that there are clouds is higher than if we don't make that assumption.

Now we will go over some motivation.

For the purpose of argument, imagine we had access to the probability distribution $\Pr$. That is, we know how likely features and classes are. For example, $\Pr(x_1 = 1)$ is the probability that the most common word, i.e. "the", is in a random movie review. Presumably, this probability is relatively high. As a second example, $\Pr(y = 1)$ is the probability that a random movie review is 'good'. In our case, this would be somewhere close to 0.67.

Since we hypothetically have access to the whole probability distribution, we also know conditional probabilities. For instance, we would know $\Pr(y = 0 \; \vert \; x_1 = 0)$, which is the probability that a random review is 'bad', given that it does not contain the word "the".

Given a probability distribution, we can find an optimal classifier which simply picks the class which maximizes the probability that we will see that class given the observed features, in other words our classifier $f: \B^n \to \B$ is given by:

$$ f(x_1, \ldots, x_n) = \argmax_{c \in \B} \Pr(y = c \given x_1, \ldots, x_n ).$$

Where $\argmax$ returns the element in $\B$ which maximizes the expression to its right, and $\B$ is the set with two elements, $\{0, 1\}$. For example, we have
$$ \argmax_{x \in \R} (x - x^2) = \frac 1 2, $$
since $\frac 1 2$ maximizes the expression $x - x^2$.

It would be great if we had access to the probability distribution $\Pr$, but unfortunately we don't in almost every case. This means we wish to try to estimate it given some samples, i.e. the training data.

However, we run into another issue: estimating the probability distribution is computionally expensive. Therefore, we assume that the different features are independent from one another. This is called the *naive conditional independence assumption*. In other words, we assume that

$$ \forall i \in \{1, \ldots, n\} : \Pr (x_i \given y, x_1, \ldots, x_{i-1}, x_{i+1}, \ldots, x_n) = \Pr(x_i \given y).$$

Using Bayes' Theorem, we can simplify the conditional independence assumption to:

$$\Pr(y \given x_1, \ldots, x_n) = \frac{\Pr(y) \prod_{i=1}^n \Pr(x_i \given y)}{\Pr(x_1, \ldots, x_n)}.$$

However, we can observe that the denominator is constant for a given input, so it's not actually necesarry to estimate it if all we want is to find the class with the maximum posterior probability. In other words,

$$ \Pr(y \given x_1, \ldots, x_n) \propto \Pr(y) \prod_{i=1}^n \Pr(x_i \given y), $$

so, our classification rule becomes

$$ f(x_1, \ldots, x_n) = \argmax_{y \in \B} \Pr(y) \prod _{i=1}^n \Pr(x_i \given y).$$

Where $\propto$ means "proportional to" and  $\prod_{i = 1}^n g(i)$ is like summation $\left(\sum_{i=1}^n g(i)\right)$, except that addition is replaced with multiplication. For example,

$$\prod_{i = 1}^5 i^2 = 1^2 \cdot 2^2 \cdot 3^2 \cdot 4^2 \cdot 5^2.$$

**Note**: To estimate prior and conditional probabilities, we use the ratios of occurence counts found in the dataset. For example, to estimate $\Pr(x_1 = 0 \; \vert \; y = 0)$, we have to calculate the number of instances of class zero for which $x_1 = 0$ and divide them by the number of instances of class 0.

**Note**: The naive independence assumption is usually false in practice for most features. Therefore, the resulting estimated probability distribution is usually a bad approximation of the true distribution. However, the resulting classifier often has a good performance, depending on the dataset.

### Quesiton 6.1 Estimating the Probability Distribution

It would be expensive to re-estimate prior and posterior probabilities every time, so we should save probabilities in memory.

Thus, you will need to save
1. $\Pr(y)$ for each $y \in \B$, and
2. $\Pr(x_i = u \; \vert \; y)$ for each $ i \in \{1, \ldots, n\}$, $u \in \mathbb{B}$ and $y \in \mathbb{B}$.

Remember that you are *estimating* the probabilities using the training set only.

In [0]:
### Answer starts here ###

def find_length(lst, n):
  only_n = [e for e in lst if e == n]

  return len(only_n)

# 1. P(y = 0)
# find the number of sample with 0 
len_y0 = find_length(y_train, 0)
len_y = len(y_train)
len_y1 = len_y - len_y0

P_y0 = len_y0/len_y
P_y1 = 1 - P_y0

# find the occurences of x_i = n_x and y = n_y
def occurences(x, y, n_x, n_y, i):
  # create a variable to store the occurences that y = n_y and x = n_x 
  tmp = 0
  # first find the number of x_i's that = n_x given that y = n_y
  for j in range(len(y)):
    if y[j] == n_y and x[j][i] == n_x:
      tmp += 1
  return tmp

# now that we have the occurences, we can calculate the probabilities of all the x_i's = {0, 1} given y = {0, 1} at each i
def probabilities(x, y, n_x, n_y, length_y):
  prob = [] 
  for i in range(len(x[0])):
    p = (occurences(x, y, n_x, n_y, i)) / length_y
    prob.append(p)
  return prob 

# x_i = 1 and y = 1
p11 = probabilities(X_train, y_train, 1, 1, len_y1) # a list with all the probabilities when x_i = 1 and y = 1
p10 = probabilities(X_train, y_train, 1, 0, len_y0)
p00 = probabilities(X_train, y_train, 0, 0, len_y0)
p01 = probabilities(X_train, y_train, 0, 1, len_y1)
### Answer ends here ###

In [0]:
print(p11[:10])
print(p01[0])

[0.8700166742458694, 0.7949825678338639, 0.747612551159618, 0.7004699105654085, 0.6089131423374261, 0.6574200394118539, 0.5993633469758981, 0.5202364711232378, 0.632787630741246, 0.6127785356980445]
0.12998332575413066


### Question 6.2 Creating the Naive Bayes Classifier

Create a function called `naive_bayes` which will take as input a list of features $x_1, \ldots, x_n$ and outputs the class with the largest posterior probability given the input features.

In [0]:
def naive_bayes(vec):
  ### Answer starts here ###

  product1 = 1 
  product0 = 1
  # first find the P(y=1)
  for i in range(len(vec)):
    if vec[i] == 1:
      product1 = product1 * p11[i]
      product0 = product0 * p10[i]
    elif vec[i] == 0:
      product1 = product1 * p01[i]
      product0 = product0 * p00[i]

  y1_given_x = P_y1 * product1  
  y0_given_x = P_y0 * product0

  if y1_given_x > y0_given_x:
    return 1
  else:
    return 0

  ### Answer ends here ###

### Question 6.3 Measuring Performance

Using the naive Bayes classifier, predict the classes for each sample point in the training set as well as the test set and print accuracies.

**Note.** You should get train and test accuracies of about 84-85%.

In [0]:
### Answer starts here ###

def prediction(data):
  prediction_vector = [] 
  for vec in data:
    prediction_vector.append(naive_bayes(vec))
  
  return prediction_vector

y_train_pred = prediction(X_train)
y_test_pred = prediction(X_test)

accuracy_train = accuracy_score(y_train, y_train_pred)
accuracy_test = accuracy_score(y_test, y_test_pred)

print(accuracy_train)
print(accuracy_test)

### Answer ends here ###

0.8346629845555896
0.8287993454694211


In [0]:
print(rfc.predict([preprocess_sample_point(
    'Boring. This movie is terrible', vocabulary)]))

print(rfc.predict([preprocess_sample_point(
    'This movie was pretty good', vocabulary)]))

[0]
[1]


## 7. Conclusion

You have successfully completed MAIS 202 assignment 2! Hope you enjoyed! To submit, download as .py and upload it to its respective okpy assignment.